In [ ]:
# Define Functions
def compare_linear(series_a, series_b, dist=lambda x, y: abs(x - y)):
    # Linear comparison requires equal-length lists
    if len(series_a) != len(series_b):
        raise IndexError("Lists must be of equal length")
    diff = 0
    for i in range(0, len(series_a)):
        diff += dist(series_a[i], series_b[i])
    return diff

def simple_dtw(series_a, series_b, dist=lambda x, y: abs(x - y), warp_weight=1):
    # DTW requires equal-length lists
    if len(series_a) != len(series_b):
        raise IndexError("Lists must be of equal length")
    if warp_weight <= 0:
        raise ValueError("Warp weight must be positive")
    # Initialize Accumulated Cost Matrix
    distances = [[0 for _ in range(len(series_a))] for _ in range(len(series_b))]
    # Initialize first cell
    distances[0][0] = dist(series_a[0], series_b[0])
    # Initialize first column
    for i in range(1, len(series_b)):
        distances[i][0] = distances[i-1][0] * warp_weight + dist(series_a[i], series_b[0])
    # Initialize first row
    for j in range(1, len(series_a)):
        distances[0][j] = distances[0][j-1] * warp_weight + dist(series_a[0], series_b[j])
    # Construct the Accumulated Cost Matrix
    for i in range(1, len(series_a)):
        for j in range(1, len(series_b)):
            distances[i][j] = min(distances[i-1][j-1], distances[i-1][j] * warp_weight, distances[i][j-1] * warp_weight) + dist(series_a[i], series_b[j])
    return distances[-1][-1], distances

In [ ]:
# Import the data
import pandas as pd

temperature = pd.read_csv("data/temperature.csv")
cities = pd.read_csv("data/city_attributes.csv")["City"].tolist()

In [ ]:
"""
Unfortunately, there are a number of missing values for 10 of the 36 cities.
The rows that are missing values are fairly consistent between these cities.
We could drop those rows, but that would leave gaps in time that would result
in the warping path crossing much larger temporal gaps than the algorithm accounts for.
Instead, we'll drop those 10 cities and compare among the remaining 26.
Each of the remaining cities are missing a value or two, but because the data
does not change rapidly from observation to observation,
this is an appropriate use-case for simple backfilling to smooth over those gaps.
"""
drop_cities = ["Vancouver", "San Francisco", "Miami", "New York", "Beersheba", "Tel Aviv District", "Eilat", "Haifa", "Nahariyya", "Jerusalem"]
temperature = temperature.drop(drop_cities, axis=1).fillna(method="bfill")
cities = [city for city in cities if city not in drop_cities]

In [ ]:
# Create City Pairs
city_pairs = []
for i in range(0, len(cities)):
    for j in range(i+1, len(cities)):
        city_pairs.append([cities[i], cities[j]])

In [ ]:
# Run Linear Comparison
for pair in city_pairs:
    pair.append(compare_linear(temperature[pair[0]], temperature[pair[1]]))
print(sorted(city_pairs, key=lambda x: x[-1]))

In [ ]:
# Don't run this! It's n squared on a very large n
"""
for pair in city_pairs:
    pair.append(simple_dtw(temperature[pair[0]], temperature[pair[1]]))
print(sorted(city_pairs, key=lambda x: x[-1]))
"""

In [ ]:
# Run This! It's linear time DTW
from fastdtw import fastdtw
for pair in city_pairs:
    pair.append(fastdtw(temperature[pair[0]], temperature[pair[1]], radius=240))
print(sorted(city_pairs[:2], key=lambda x: x[-1]))

In [ ]:
# Bonus: Compare simple_dtw() function to dtw library on small datasets
from dtw import dtw
import random

for _ in range(100):
    x = []
    y = []
    for _ in range(100):
        x.append(random.randint(0, 10))
        y.append(random.randint(0, 10))
    d, cost_matrix, acc_cost_matrix, path = dtw(x, y, dist=lambda x, y: abs(x - y), s=2)
    s_d, s_acc_cost_matrix = simple_dtw(x, y, dist=lambda x, y: abs(x - y), warp_weight=2)
    assert(d == s_d)

print("simple_dtw passed equivalence test")